In [ ]:
from huggingface_hub import login
login()

In [ ]:
!wandb login --relogin

In [ ]:
import os
import wandb


# CONFIG
LOCAL_DIR = "/content/checkpoints"
os.makedirs(LOCAL_DIR, exist_ok=True)

# WandB run ID that contains your checkpoint files
RUN_PATH = "eren23/blip2-llama-livis-50k/i7hxfev8"

# Filenames inside the run
QFORMER_FILE   = "checkpoints/qformer_step10k.pt"
PROJECTOR_FILE = "checkpoints/projector_step10k.pt"


# Download from run files API
print("🔵 Connecting to W&B API...")
api = wandb.Api()

run = api.run(RUN_PATH)

def download_run_file(filename, local_dir):
    print(f"⬇ Downloading {filename} ...")
    f = run.file(filename)
    f.download(root=local_dir, replace=True)
    local_path = os.path.join(local_dir, os.path.basename(filename))
    print(f"✔ Saved to: {local_path}")
    return local_path

q_path = download_run_file(QFORMER_FILE, LOCAL_DIR)
p_path = download_run_file(PROJECTOR_FILE, LOCAL_DIR)

print("\n✅ Downloads complete:")
print("Q-Former  →", q_path)
print("Projector →", p_path)


In [ ]:

# BLIP-2 Inference + Batch Attention Visualizations

import os, math, torch
from torch import nn
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoProcessor,
    SiglipVisionModel,
)


# CONFIG

device = "cuda"
torch.backends.cuda.matmul.allow_tf32 = True

LLM_NAME    = "meta-llama/Llama-3.2-1B"
VISION_NAME = "google/siglip-so400m-patch14-384"

# Point these to the weights you trained (e.g. step10k or best)
QFORMER_CKPT  = "/content/checkpoints/checkpoints/qformer_step10k.pt"
PROJECTOR_CKPT = "/content/checkpoints/checkpoints/projector_step10k.pt"

ATTN_OUT_DIR = "/content/attn_viz"
os.makedirs(ATTN_OUT_DIR, exist_ok=True)

PROMPT_PREFIX = "Short caption: "

print("Device:", device)


# LOAD LLM + TOKENIZER

print("🧠 Loading LLaMA…")
tokenizer = AutoTokenizer.from_pretrained(LLM_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

llm = AutoModelForCausalLM.from_pretrained(
    LLM_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
)
llm.eval()
d_model = llm.config.hidden_size
print("LLM hidden size:", d_model)


# 2) LOAD SIGLIP VISION ENCODER

print("👁 Loading SigLIP…")
processor = AutoProcessor.from_pretrained(VISION_NAME)
vision = SiglipVisionModel.from_pretrained(
    VISION_NAME,
    torch_dtype=torch.float16,
).to(device)
vision.eval()
d_vis = vision.config.hidden_size
print("SigLIP hidden size:", d_vis)


# 3) Q-FORMER + PROJECTOR (MATCH TRAINING ARCH)
#    - WITH ATTENTION STORAGE

class QFormerBlock(nn.Module):
    """
    One BLIP-2 style Q-Former block:
      - LN + self-attn on queries
      - LN + cross-attn (queries -> vision tokens)
      - LN + MLP
    """
    def __init__(self, d_model, n_heads, mlp_ratio=4.0, dropout=0.1):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.self_attn = nn.MultiheadAttention(
            d_model, n_heads, batch_first=True, dropout=dropout
        )

        self.ln2 = nn.LayerNorm(d_model)
        self.cross_attn = nn.MultiheadAttention(
            d_model, n_heads, batch_first=True, dropout=dropout
        )

        self.ln3 = nn.LayerNorm(d_model)
        hidden = int(d_model * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(d_model, hidden),
            nn.GELU(),
            nn.Linear(hidden, d_model),
            nn.Dropout(dropout),
        )

    def forward(self, q, v):
        # q, v: (B, K, d), (B, N, d)
        # self-attention
        q2, _ = self.self_attn(
            self.ln1(q), self.ln1(q), self.ln1(q),
            need_weights=False
        )
        q = q + q2

        # cross-attention: queries = q, keys/values = v
        q2, attn = self.cross_attn(
            self.ln2(q), self.ln2(v), self.ln2(v),
            need_weights=True,
            average_attn_weights=False,  # (B, heads, K, N)
        )
        q = q + q2

        # MLP
        q = q + self.mlp(self.ln3(q))
        return q, attn


class QFormer(nn.Module):
    def __init__(self, d_vis, d_model,
                 n_queries=32,
                 n_layers=8,
                 heads=8,
                 mlp_ratio=4.0,
                 dropout=0.1):
        super().__init__()
        self.query = nn.Parameter(torch.randn(n_queries, d_model))
        self.vis_proj = nn.Linear(d_vis, d_model)
        self.layers = nn.ModuleList([
            QFormerBlock(d_model, heads, mlp_ratio=mlp_ratio, dropout=dropout)
            for _ in range(n_layers)
        ])
        self.final_ln = nn.LayerNorm(d_model)
        self.last_attn = None  # (B, heads, K, N) from last block

    def forward(self, vis_tokens):
        # vis_tokens: (B, N, d_vis)
        v = self.vis_proj(vis_tokens.to(torch.float32))   # (B, N, d_model)
        B = v.size(0)
        q = self.query.unsqueeze(0).expand(B, -1, -1)     # (B, K, d_model)

        last_attn = None
        for blk in self.layers:
            q, attn = blk(q, v)
            last_attn = attn

        self.last_attn = last_attn  # (B, heads, K, N)
        return self.final_ln(q)     # (B, K, d_model)


class Projector(nn.Module):
    """
    Strong projector used in training:
      y = LN( x + MLP(LN(x)) )
    """
    def __init__(self, d_model, mlp_ratio=4.0, dropout=0.1):
        super().__init__()
        hidden = int(d_model * mlp_ratio)
        self.ln1 = nn.LayerNorm(d_model)
        self.mlp = nn.Sequential(
            nn.Linear(d_model, hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, d_model),
            nn.Dropout(dropout),
        )
        self.ln2 = nn.LayerNorm(d_model)

    def forward(self, x):
        r = x
        x = self.ln1(x)
        x = self.mlp(x)
        x = r + x
        return self.ln2(x)


qformer = QFormer(d_vis, d_model).to(device)
projector = Projector(d_model).to(device)

print("📥 Loading Q-Former weights from:", QFORMER_CKPT)
qformer.load_state_dict(torch.load(QFORMER_CKPT, map_location=device))
print("📥 Loading Projector weights from:", PROJECTOR_CKPT)
projector.load_state_dict(torch.load(PROJECTOR_CKPT, map_location=device))

qformer.eval()
projector.eval()


# SMALL UTILS: TOKEN GRID SHAPE

def tokens_to_grid(attn_1d: torch.Tensor):
    """
    Convert 1D attention over N tokens → (H, W) grid.
    Picks H,W so H*W divides N.
    """
    N = attn_1d.numel()
    h = int(math.sqrt(N))
    while h > 1 and (N % h) != 0:
        h -= 1
    w = N // h
    return attn_1d[:h * w].reshape(h, w)



# CORE ENCODING (VISION + QFORMER + PROJECTOR)

@torch.no_grad()
def encode_batch(images):
    """
    images: list[ PIL.Image ]
    returns:
      q_proj: (B, K, d_model) projected queries
      attn:   (B, heads, K, N) last cross-attn maps
    """
    imgs = [im.convert("RGB") for im in images]
    v = processor(images=imgs, return_tensors="pt")["pixel_values"].to(device)

    vout = vision(pixel_values=v)
    vtoks = vout.last_hidden_state[:, 1:, :]   # drop CLS → (B, N, d_vis)

    q = qformer(vtoks)                         # (B, K, d_model)
    attn = qformer.last_attn                  # (B, heads, K, N)

    q_proj = projector(q).to(llm.dtype)        # (B, K, d_model)
    return q_proj, attn



# BATCH GENERATION

@torch.no_grad()
def infer_with_attention(
    images,
    batch_tag="batch0",
    max_new_tokens=64,
    num_queries_vis=4
):
    """
    images: list[ PIL.Image ]
    Returns:
      captions: list[str]
      attn_paths: dict with keys:
          'per_query_grid', 'per_image_accum_grid',
          'per_image_single_accum_paths' (list of per-image pngs)
    Also saves multiple visualization PNGs under ATTN_OUT_DIR.
    """
    B = len(images)
    assert B > 0

    q_proj, attn = encode_batch(images)  # q_proj: (B,K,d), attn: (B,heads,K,N)
    B, heads, K, N = attn.shape

    # Text prefix for all samples
    prompts = [PROMPT_PREFIX] * B
    tok = tokenizer(prompts, return_tensors="pt", padding=True).to(device)
    txt_emb = llm.get_input_embeddings()(tok["input_ids"])   # (B, T, d)
    T = txt_emb.size(1)

    # Build prefix+text embeddings
    full_emb = torch.cat([q_proj, txt_emb], dim=1)           # (B, K+T, d)
    attn_mask = torch.ones(B, K + T, device=device, dtype=torch.long)

    gen_ids = llm.generate(
        inputs_embeds=full_emb,
        attention_mask=attn_mask,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id,
    )

    # Decode + strip prompt
    captions = []
    for i in range(B):
        text = tokenizer.decode(gen_ids[i], skip_special_tokens=True)
        if text.startswith(PROMPT_PREFIX):
            text = text[len(PROMPT_PREFIX):].strip()
        captions.append(text)

    # Save attention visualizations
    attn_paths = save_attention_visuals(
        images, attn, batch_tag=batch_tag, num_queries_vis=num_queries_vis
    )

    return captions, attn_paths


# ATTENTION VISUALIZATION (WITH UPSAMPLING)

import torch.nn.functional as F

def upsample_to_image(grid: np.ndarray, img_np: np.ndarray):
    """
    Upsample a (h×w) attention grid to match (H×W) of the image.
    Uses bilinear interpolation for smooth scaling.
    """
    h, w = grid.shape
    H, W = img_np.shape[:2]

    t = torch.from_numpy(grid).float().unsqueeze(0).unsqueeze(0)  # (1,1,h,w)
    t_up = F.interpolate(t, size=(H, W), mode="bilinear", align_corners=False)
    return t_up.squeeze().numpy()  # (H, W)


def save_attention_visuals(images, attn, batch_tag="batch0", num_queries_vis=4):
    """
    images: list[ PIL.Image ]
    attn:   (B, heads, K, N)

    Saves:
      - per-query grid PNG
      - per-image accumulated PNGs
      - batch accumulated PNG
    All now use upsampled attention maps for better spatial alignment.
    """
    B = len(images)
    attn = attn.detach().cpu()     # (B, H, K, N)
    heads = attn.size(1)
    K = attn.size(2)
    N = attn.size(3)

    num_q = min(num_queries_vis, K)

    # Average attention across heads → (B, K, N)
    attn_q = attn.mean(dim=1)

    # Per-image grid: [original | per-query heatmaps]
    fig, axes = plt.subplots(
        B, num_q + 1,
        figsize=(3 * (num_q + 1), 3 * B),
        squeeze=False
    )

    for b in range(B):
        img_np = np.array(images[b].convert("RGB"))

        # Column 0: original image
        axes[b, 0].imshow(img_np)
        axes[b, 0].set_title(f"Img {b}")
        axes[b, 0].axis("off")

        # Each query slot
        for qi in range(num_q):
            a_1d = attn_q[b, qi]                  # (N,)
            grid = tokens_to_grid(a_1d)           # (h,w)
            g = grid.numpy()
            g = (g - g.min()) / (g.max() - g.min() + 1e-8)

            # 🔥 Upsample to match full image resolution
            g_up = upsample_to_image(g, img_np)

            ax = axes[b, qi + 1]
            ax.imshow(img_np)
            ax.imshow(g_up, cmap="jet", alpha=0.42)
            ax.set_title(f"Q{qi}")
            ax.axis("off")

    plt.tight_layout()
    per_query_path = os.path.join(ATTN_OUT_DIR, f"attn_queries_{batch_tag}.png")
    plt.savefig(per_query_path, dpi=120)
    plt.close(fig)

    # Accumulated attention across queries + heads
    # attn: (B, heads, K, N) → avg heads+queries → (B, N)
    attn_accum = attn.mean(dim=1).mean(dim=1)

    fig2, axes2 = plt.subplots(B, 2, figsize=(6, 3 * B), squeeze=False)
    per_image_accum_paths = []

    for b in range(B):
        img_np = np.array(images[b].convert("RGB"))
        a_1d = attn_accum[b]                  # (N,)
        grid = tokens_to_grid(a_1d)
        g = grid.numpy()
        g = (g - g.min()) / (g.max() - g.min() + 1e-8)

        # 🔥 Upsample accumulated attention
        g_up = upsample_to_image(g, img_np)

        # Batch grid (left = original, right = accumulated)
        axes2[b, 0].imshow(img_np)
        axes2[b, 0].set_title(f"Img {b}")
        axes2[b, 0].axis("off")

        axes2[b, 1].imshow(img_np)
        axes2[b, 1].imshow(g_up, cmap="jet", alpha=0.42)
        axes2[b, 1].set_title("Accum")
        axes2[b, 1].axis("off")

        # Per-image standalone PNG
        fig_single, ax_single = plt.subplots(1, 2, figsize=(6, 3))

        ax_single[0].imshow(img_np)
        ax_single[0].set_title("Original")
        ax_single[0].axis("off")

        ax_single[1].imshow(img_np)
        ax_single[1].imshow(g_up, cmap="jet", alpha=0.42)
        ax_single[1].set_title("Accumulated")
        ax_single[1].axis("off")

        plt.tight_layout()
        single_path = os.path.join(
            ATTN_OUT_DIR, f"attn_accum_img{b}_{batch_tag}.png"
        )
        plt.savefig(single_path, dpi=120)
        plt.close(fig_single)
        per_image_accum_paths.append(single_path)

    plt.tight_layout()
    accum_grid_path = os.path.join(
        ATTN_OUT_DIR, f"attn_accum_grid_{batch_tag}.png"
    )
    plt.savefig(accum_grid_path, dpi=120)
    plt.close(fig2)

    return {
        "per_query_grid": per_query_path,
        "per_image_accum_grid": accum_grid_path,
        "per_image_accum_paths": per_image_accum_paths,
    }


# SIMPLE SINGLE-IMAGE HELPER

@torch.no_grad()
def generate_caption(img: Image.Image, max_new_tokens=128):
    caps, _ = infer_with_attention([img], batch_tag="single", max_new_tokens=max_new_tokens)
    return caps[0]


print("✅ Inference + attention system ready.")
print(f"Attention PNGs will be saved under: {ATTN_OUT_DIR}")




In [ ]:

# from glob import glob
# test_paths = sorted(glob("/content/livis_cache/*.jpg"))[:15]
# imgs = [Image.open(p) for p in test_paths]
# captions, paths = infer_with_attention(imgs, batch_tag="test_batch", max_new_tokens=64)
# print("Captions:", captions)
# print("Saved attention:", paths)

In [ ]:
imgs = [Image.open("muco.png")]
infer_with_attention(imgs, batch_tag="test_batch", max_new_tokens=64)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


(["\xa0A person in a yellow life jacket sits on a yellow boat near a rocky shoreline, surrounded by trees and a clear sky, indicating a recreational activity or training session. The person wears a helmet and holds a yellow paddle, while the boat is stationary. The sky is clear, and the person's face is visible, indicating"],
 {'per_query_grid': '/content/attn_viz/attn_queries_test_batch.png',
  'per_image_accum_grid': '/content/attn_viz/attn_accum_grid_test_batch.png',
  'per_image_accum_paths': ['/content/attn_viz/attn_accum_img0_test_batch.png']})